In [1]:
#Refine FFTDock poses via minimization in explicit protein
#Inputs:
    #cofactor/pdb_with_fad/tropb.pdb
    #dock/ligands/2.pdb
    #dock/ligands/2.str #CGenFF stream file
    #dock/poses/tropb_2_fftdock
    #toppar
#Outputs:
    #dock/poses/tropb_2_prot
    #dock/scores/tropb_2_prot.csv

In [2]:
import os 
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
os.environ['CHARMM_LIB_DIR'] = "/home/azamh/charmm/c48a/pycharmm_build/install/lib/"

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake
import pycharmm.settings as settings
import pycharmm.grid as grid
import pycharmm.charmm_file as charmm_file
from pycharmm.select_atoms import SelectAtoms
from pycharmm.lingo import charmm_script
from pycharmm.lib import charmm as libcharmm

In [3]:
#Arguments
protein = 'tropb'
ligand = '2'
toppardir = '../../toppar'
liganddir = '../ligands'
proteindir = '../../cofactor/pdb_with_fad'
fftdockdir = f'../poses/{protein}_{ligand}_fftdock'
dockdir = f'../poses/{protein}_{ligand}_prot'
os.makedirs(dockdir, exist_ok=True)

In [4]:
## Read in the topology and parameter file 
settings.set_bomb_level(-1)
read.rtf(os.path.join(toppardir, 'top_all36_prot.rtf'))
read.rtf(os.path.join(toppardir,'top_all36_cgenff.rtf'), append = True)
read.rtf(os.path.join(toppardir,'probes.rtf'), append = True)
read.prm(os.path.join(toppardir, 'par_all36m_prot.prm'), flex = True)
read.prm(os.path.join(toppardir, 'par_all36_cgenff.prm'), append = True, flex = True)
read.prm(os.path.join(toppardir, 'probes.prm'), append = True, flex = True)
settings.set_bomb_level(0)
charmm_script(f'stream {os.path.join(liganddir, ligand)}.str')
charmm_script(f'stream {os.path.join(toppardir, "st2_fadh.str")}')

  
 CHARMM>     read rtf card -
 CHARMM>     name ../../toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::../../toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read rtf card -
 CHARMM>     name ../../toppar/top_all36_cgenff.rtf -
 CHARMM>     append
 VOPEN> Attempting to open::../../toppar/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD 

1

In [5]:
#Build ligand
ligand_pdb = os.path.join(liganddir, f'{ligand}.pdb')
read.sequence_pdb(ligand_pdb)
gen.new_segment(seg_name = "LIGA")
read.pdb(ligand_pdb, resid = True)

  
 CHARMM>     read sequence pdb -
 CHARMM>     name ../ligands/2.pdb
 VOPEN> Attempting to open::../ligands/2.pdb::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     1 RESIDUES
          LIG 
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   1 has been generated. Its identifier is LIGA.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        1   Number of residues   =        1
         Number of atoms         =       32   Number of groups     =        1
         Number of bonds         =       32   Number of angles     =       55
         Number of dihedrals     =       72   Number of impropers  =        2
         Number of cross-terms   =        0   Num

In [6]:
#Setup nonbonds
my_nbonds = pycharmm.NonBondedScript(
    cutnb=12.0, ctonnb=10.0, ctofnb=10.0,
    eps=0.75,
    cdie=False,
    rdie=True,
    switch=True, vswitch=True)
# Implement these non-bonded parameters by "running" them.
my_nbonds.run()

  
 CHARMM>     nbonds cutnb 12.0 -
 CHARMM>     ctonnb 10.0 -
 CHARMM>     ctofnb 10.0 -
 CHARMM>     eps 0.75 -
 CHARMM>     rdie -
 CHARMM>     switch -
 CHARMM>     vswitch

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    RDIElec  SWITch   VATOm    VSWItch 
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 12.000 CTEXNB =999.000 CTONNB = 10.000 CTOFNB = 10.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  0.750
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      12.0      10.0      10.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
      ***** CUTNB,CTOFNB,CTONNB are not in correct order.
      ******************************************
      BOMLEV (  0) IS NOT REACHED. WRNLEV IS  5

 <MAKINB> with mode   5 found     87 exclusions and     69 interactions(1-4)
 <MAKGRP> found      0 group exclusions.
 Generat

In [7]:
#Minimize ligand in vacuum for initial energy
minimize.run_sd(nstep=1000, tolenr=1e-3, tolgrd=1e-4)


 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    RDIElec  SWITch   VATOm    VSWItch 
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 12.000 CTEXNB =999.000 CTONNB = 10.000 CTOFNB = 10.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  0.750
 NBXMOD =      5
 There are      409 atom  pairs and      156 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      12.0      10.0      10.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
      ***** CUTNB,CTOFNB,CTONNB are not in correct order.
      ******************************************
      BOMLEV (  0) IS NOT REACHED. WRNLEV IS  5

 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR      481 ATOM PAIRS AND        0 GROUP PAIRS

 General atom nonbond list generation found:
      409 ATOM PAIRS WERE FOUND FOR ATOM LIST
        1 GROUP PAIRS REQUIRED ATOM SEARCHES

 PRNHBD: CUToff Hydrogen Bond  distance =    0

True

 ----------       ---------    ---------    ---------    ---------    ---------


In [8]:
## Build protein
protein_psf = os.path.join(proteindir, f'{protein}_fad.psf')
protein_pdb = os.path.join(proteindir, f'{protein}_fad.pdb')
read.psf_card(protein_psf, append = True)
read.pdb(protein_pdb, resid = True)

  
 CHARMM>     read psf card -
 CHARMM>     name ../../cofactor/pdb_with_fad/tropb_fad.psf -
 CHARMM>     append
 VOPEN> Attempting to open::../../cofactor/pdb_with_fad/tropb_fad.psf::
 MAINIO> Protein structure file being appended from unit  91.
 psf_read_formatted: Reading PSF in the expanded format.
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *  DATE:     6/22/23     21:37:41      CREATED BY USER: azamh
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        3   Number of residues   =      449
         Number of atoms         =     7072   Number of groups     =     2043
         Number of bonds         =     7162   Number of angles     =    12887
         Number of dihedrals     =    18839   Number of impropers  =     1277
         Number of cross-terms   =      447   Number of autogens   =        0
         Number of HB acceptors  =      652   Number of HB donor

In [9]:
#Fix protein and cofactor atoms
cons_fix.setup(selection = ~SelectAtoms(seg_id='LIGA'))

True

In [10]:
#Get initial energy of system by translating ligand away from protein
charmm_script('coor tranlate xdir 400 ydir 400 zdir 400 select segid LIGA end')

  
 CHARMM>     coor tranlate xdir 400 ydir 400 zdir 400 select segid LIGA end
 SELRPN>     32 atoms have been selected out of   7072
 TRANSLATION VECTOR   400.000000  400.000000  400.000000
 SELECTED COORDINATES TRANSLATED IN THE MAIN SET.

  


1

In [11]:
#Get initial energy
def get_energy_df(pose_name):
    df = energy.get_energy()
    df = df[df.columns[0:10]]
    df.index = [pose_name]
    df.index.name = 'pose'
    df.columns.name = 'term'
    return df
initial_energy_df = get_energy_df('initial')
initial_energy_df


 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    RDIElec  SWITch   VATOm    VSWItch 
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 12.000 CTEXNB =999.000 CTONNB = 10.000 CTOFNB = 10.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  0.750
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      12.0      10.0      10.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
      ***** CUTNB,CTOFNB,CTONNB are not in correct order.
      ******************************************
      BOMLEV (  0) IS NOT REACHED. WRNLEV IS  5

 <MAKINB> with mode   5 found  20049 exclusions and  18468 interactions(1-4)
 <MAKGRP> found   5960 group exclusions.
 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR  1279721 ATOM PAIRS AND        0 GROUP PAIRS
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is

term,ENER,GRMS,DELTA,BOND,ANGL,UREY,DIHE,IMPR,VDW,ELEC
pose,,,,,,,,,,
initial,-14.885048,1.834568,0.0,2.273115,3.203081,0.216589,6.4627,0.009944,14.654264,-41.704742


In [12]:
#Minimize all fftdock poses
nsave = 500
pose_energy_dfs = [initial_energy_df]

#Hide large output
settings.set_verbosity(0)
settings.set_warn_level(-2)
for i in tqdm(range(1, nsave + 1)):
    
    #Read FFTDock pose
    fftdock_pose = os.path.join(fftdockdir, f'{protein}_{ligand}_{i}.crd')
    read.pdb(fftdock_pose, resid=True)
    energy.show()
    
    #Perform minimization in explicit protein
    minimize.run_sd(nstep=50)
    minimize.run_abnr(nstep=1000, tolenr = 1e-3)

    #Get refined energy
    pose_energy_df = get_energy_df(i)
    pose_energy_dfs.append(pose_energy_df)

    #write pdb
    pose_pdb = os.path.join(dockdir, f'{protein}_{ligand}_{i}.pdb')
    write.coor_pdb(pose_pdb, sele = 'segid LIGA end')

settings.set_verbosity(5)
settings.set_warn_level(0)

100%|██████████| 500/500 [03:07<00:00,  2.67it/s]


-2

In [13]:
#Concat energy dataframes
energy_df = pd.concat(pose_energy_dfs).fillna(0)
energy_df

term,ENER,GRMS,DELTA,BOND,ANGL,UREY,DIHE,IMPR,VDW,ELEC
pose,,,,,,,,,,
initial,-14.885048,1.834568,0.000000,2.273115,3.203081,0.216589,6.462700,0.009944,14.654264,-41.704742
1,95.449755,1.834568,-110.334803,4.838610,24.010097,1.901835,65.514666,0.008773,64.861330,-65.685556
2,395.224657,1.834568,-410.109705,87.977588,53.120644,7.126278,46.333686,0.051991,293.460331,-92.845861
3,28.407252,1.834568,-43.292300,2.456419,14.823023,0.722913,67.395853,0.068396,15.381243,-72.440594
4,-1.469720,1.834568,-13.415328,2.344571,6.273513,0.588621,19.047017,0.422566,34.782854,-64.928862
...,...,...,...,...,...,...,...,...,...,...
496,6.807839,1.834568,-21.692887,2.396042,15.049832,1.374329,37.380701,0.028626,16.563998,-65.985689
497,24.801323,1.834568,-39.686371,3.882458,13.682915,1.112925,26.363073,0.240655,33.638162,-54.118865
498,16.627800,1.834568,-31.512848,2.664677,10.416114,0.554268,52.834797,0.179688,24.790469,-74.812214


In [14]:
#Get final-initial energy
delta_energy_df = energy_df.subtract(energy_df.loc['initial'].values, axis = 1)
delta_energy_df

term,ENER,GRMS,DELTA,BOND,ANGL,UREY,DIHE,IMPR,VDW,ELEC
pose,,,,,,,,,,
initial,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,110.334803,0.0,-110.334803,2.565495,20.807015,1.685246,59.051966,-0.001171,50.207066,-23.980814
2,410.109705,0.0,-410.109705,85.704473,49.917563,6.909689,39.870986,0.042047,278.806066,-51.141120
3,43.292300,0.0,-43.292300,0.183304,11.619941,0.506324,60.933153,0.058452,0.726978,-30.735852
4,13.415328,0.0,-13.415328,0.071456,3.070431,0.372032,12.584316,0.412622,20.128590,-23.224120
...,...,...,...,...,...,...,...,...,...,...
496,21.692887,0.0,-21.692887,0.122927,11.846750,1.157740,30.918001,0.018682,1.909733,-24.280947
497,39.686371,0.0,-39.686371,1.609343,10.479834,0.896336,19.900373,0.230711,18.983897,-12.414123
498,31.512848,0.0,-31.512848,0.391562,7.213033,0.337679,46.372097,0.169744,10.136205,-33.107472


In [15]:
#Save energies
scorefile = f'../scores/{protein}_{ligand}_prot.csv'
delta_energy_df.to_csv(scorefile)